In [1]:
#!pip install networkx
#!pip install python-louvain
#!pip3 install networkit




In [11]:
import networkx as nx
import matplotlib.pyplot as plt
from multiprocessing import Pool
import itertools
import community
import pandas as pd

import networkit as nk 


## Finding centrality of source twitter and target twitter account


In [12]:
df = pd.read_csv('social_network_data_2021_1227.csv')
df = df.dropna(subset=['source_twitter', 'target_twitter'])
print(df.shape)
df.head()

(258764, 16)


,source_twitter,target_twitter,source_location,target_location,follow_actions,interaction_times,interaction_type_counts,customer_type,tier,a_level,age_range,gender,orders,amount,first_authorized_at,first_payment_date
0,0120iesagashi,109btt,BTT,NaN,1,0,0,customer_registered_only,1-Bronze,a_level_0,not available,not available,NaN,NaN,2021-08-17 00:00:00.000,NaN
1,0120iesagashi,16toyosu,BTT,NaN,1,0,0,customer_registered_only,1-Bronze,a_level_0,not available,not available,NaN,NaN,2021-08-17 00:00:00.000,NaN
2,0120iesagashi,1manken,BTT,NaN,1,0,0,customer_registered_only,1-Bronze,a_level_0,not available,not available,NaN,NaN,2021-08-17 00:00:00.000,NaN
3,0120iesagashi,2020wngn,BTT,NaN,1,1,1,customer_registered_only,1-Bronze,a_level_0,not available,not available,NaN,NaN,2021-08-17 00:00:00.000,NaN
4,0120iesagashi,2022Btt,BTT,NaN,0,1,1,customer_registered_only,1-Bronze,a_level_0,not available,not available,NaN,NaN,2021-08-17 00:00:00.000,NaN


In [13]:
#df[['source_twitter', 'target_twitter']][:3000].to_csv('3k_accs.csv', index=False)

In [14]:
len(df['source_twitter'].unique()) + len(df['target_twitter'].unique()) 

178571

### Read with `NetworkX`

### Betweeness centrality: Finding campaign targets
#### A node is important if it lies on many shortest paths between other nodes

In [36]:
nxG = nx.from_pandas_edgelist(df, 
                            source='source_twitter',
                            target='target_twitter',
                            edge_attr=True,
                            create_using=nx.Graph())

print(nx.info(nxG))



Name: 
Type: Graph
Number of nodes: 178438
Number of edges: 258757
Average degree:   2.9002


In [40]:


G = nk.nxadapter.nx2nk(nxG)

# Get mapping from node ids in nxG to node ids in G
idmap = dict((id, u) for (id, u) in zip(nxG.nodes(), range(nxG.number_of_nodes())))

inverse_map = {v: k for k, v in idmap.items()}
print(type(inverse_map))
inverse_map_df = pd.DataFrame(inverse_map.items(), columns=['Index', 'Twitter'])
inverse_map_df

<class 'dict'>


,Index,Twitter
0,0,0120iesagashi
1,1,109btt
2,2,16toyosu
3,3,1manken
4,4,2020wngn
...,...,...
178433,178433,yuru_creation
178434,178434,yuzriha_h
178435,178435,yuzutouhu_ika
178436,178436,zaru_cr


In [44]:
closeness_centrality = nk.centrality.Closeness(G,False,1)
closeness_centrality.run()

In [50]:
closeness_centrality_df = pd.DataFrame(dict(closeness_centrality.ranking()).items(), columns=['Twitter_CC','Closeness Centrality'])
closeness_centrality_df

,Twitter_CC,Closeness Centrality
0,603,0.343266
1,4739,0.337610
2,84400,0.332801
3,22271,0.332541
4,93694,0.330895
...,...,...
178433,71314,0.154203
178434,160101,0.000017
178435,160102,0.000010
178436,160103,0.000010


In [51]:
closeness_centrality_df = closeness_centrality_df.merge(inverse_map_df,
                                                        left_on=closeness_centrality_df['Twitter_CC'], 
                                                        right_on=inverse_map_df['Index'])

In [56]:
closeness_centrality_df = closeness_centrality_df[['Twitter', 'Closeness Centrality']]
closeness_centrality_df = closeness_centrality_df.sort_values(by='Closeness Centrality', ascending=False)
closeness_centrality_df = closeness_centrality_df.rename(columns={'Twitter': 'Twitter_CC'})

closeness_centrality_df

,Twitter_CC,Closeness Centrality
0,yousuck2020,0.343266
1,xnoahxbabyx,0.337610
2,iloveyou1984022,0.332801
3,KOICHIOSHIMA2,0.332541
4,kenzi7642,0.330895
...,...,...
178432,youyuxi,0.154203
178434,tag60065904,0.000017
178435,O21Otg,0.000010
178436,_O913ykh_,0.000010


In [57]:
#closeness_centrality_df.to_csv('closeness_centrality.csv', index=False)